# 🧮 Phase 1 — Bioeconomic LP (Single-Year) + Validator
This notebook solves a single-year LP and validates against **Table 10 & 11** values (Birr/HH/year).

> Edit CSVs in `./data/` to try different assumptions later; re-run the last cells to update results + validation.

In [1]:
import json
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.optimize import linprog
# from caas_jupyter_tools import display_dataframe_to_user
import ace_tools_open as tools

print('Loaded: numpy, pandas, scipy.optimize.linprog')

Loaded: numpy, pandas, scipy.optimize.linprog


In [2]:
@dataclass
class HouseholdClass:
    name: str
    n_households: float
    adult_equiv: float
    labor_endowment: float
    land_available: float
    max_hired_labor: float

@dataclass
class CropParam:
    name: str
    calorie_per_kg: float
    yield_per_ha: float
    price_sale: float
    seed_cost_per_ha: float
    fert_cost_per_ha: float
    chem_cost_per_ha: float
    labor_req_per_ha: float

@dataclass
class LivestockParam:
    name: str
    price_sale: float
    feed_cost_per_unit: float
    vet_cost_per_unit: float
    labor_req_per_unit: float

@dataclass
class PriceParam:
    wage: float

@dataclass
class ScenarioShocks:
    yield_multiplier: float = 1.0
    crop_price_multiplier: float = 1.0
    wage_multiplier: float = 1.0
    fert_price_multiplier: float = 1.0
    population_multiplier: float = 1.0

@dataclass
class ModelParams:
    households: Dict[str, HouseholdClass]
    crops: Dict[str, CropParam]
    livestock: Dict[str, LivestockParam]
    prices: PriceParam
    min_kcal_per_person_per_day: float = 2000.0
    days_per_year: int = 365

print('Schemas defined.')

Schemas defined.


In [3]:
DATA_DIR = Path('./data')

# Show what's currently in data/ for easy editing
try:
    tools.display_dataframe_to_user('households.csv', pd.read_csv(DATA_DIR / 'households.csv'))
    tools.display_dataframe_to_user('crops.csv', pd.read_csv(DATA_DIR / 'crops.csv'))
    tools.display_dataframe_to_user('livestock.csv', pd.read_csv(DATA_DIR / 'livestock.csv'))
    tools.display_dataframe_to_user('prices.csv', pd.read_csv(DATA_DIR / 'prices.csv'))
    tools.display_dataframe_to_user(
        "observed_table10_11.csv", pd.read_csv(DATA_DIR / "observed_table10_11.csv")
    )
except Exception:
    pass
print('Templates loaded. You can edit any of them and re-run the solver below.')

households.csv


Loading ITables v2.5.2 from the internet... (need help?)


crops.csv


Loading ITables v2.5.2 from the internet... (need help?)


livestock.csv


...

prices.csv


Loading ITables v2.5.2 from the internet... (need help?)


observed_table10_11.csv


Loading ITables v2.5.2 from the internet... (need help?)


Templates loaded. You can edit any of them and re-run the solver below.


In [4]:
def load_params_from_folder(folder: Path) -> Tuple[ModelParams, ScenarioShocks]:
    hh = pd.read_csv(folder / 'households.csv')
    crops = pd.read_csv(folder / 'crops.csv')
    livest = pd.read_csv(folder / 'livestock.csv')
    prices_df = pd.read_csv(folder / 'prices.csv')
    with open(folder / 'scenario.json', 'r') as f:
        sc_dict = json.load(f)

    households = {r['name']: HouseholdClass(
        name=r['name'], n_households=float(r['n_households']), adult_equiv=float(r['adult_equiv']),
        labor_endowment=float(r['labor_endowment']), land_available=float(r['land_available']),
        max_hired_labor=float(r.get('max_hired_labor', 0.0))
    ) for _, r in hh.iterrows()}

    crops_map = {r['name']: CropParam(
        name=r['name'], calorie_per_kg=float(r['calorie_per_kg']), yield_per_ha=float(r['yield_per_ha']),
        price_sale=float(r['price_sale']), seed_cost_per_ha=float(r['seed_cost_per_ha']),
        fert_cost_per_ha=float(r['fert_cost_per_ha']), chem_cost_per_ha=float(r['chem_cost_per_ha']),
        labor_req_per_ha=float(r['labor_req_per_ha'])
    ) for _, r in crops.iterrows()}

    livest_map = {r['name']: LivestockParam(
        name=r['name'], price_sale=float(r['price_sale']), feed_cost_per_unit=float(r['feed_cost_per_unit']),
        vet_cost_per_unit=float(r['vet_cost_per_unit']), labor_req_per_unit=float(r['labor_req_per_unit'])
    ) for _, r in livest.iterrows()}

    prices = PriceParam(wage=float(prices_df.iloc[0]['wage']))
    scenario = ScenarioShocks(**sc_dict)
    params = ModelParams(households=households, crops=crops_map, livestock=livest_map, prices=prices)
    return params, scenario


def build_index_maps(H, C, L):
    idx = {}; pos = 0
    for h in H:
        for c in C: idx[("area", h, c)] = pos; pos += 1
    for h in H:
        for c in C: idx[("cons", h, c)] = pos; pos += 1
    for h in H:
        for c in C: idx[("sold", h, c)] = pos; pos += 1
    for h in H:
        for c in C: idx[("stored", h, c)] = pos; pos += 1
    for h in H: idx[("hired", h, None)] = pos; pos += 1
    for h in H: idx[("off_farm", h, None)] = pos; pos += 1
    for h in H:
        for l in L: idx[("live_units", h, l)] = pos; pos += 1
    return idx, pos


def solve_single_year_lp(params: ModelParams, scenario: ScenarioShocks, hh_subset: Optional[List[str]] = None, calories_floor_override: Optional[float] = None):
    H = hh_subset if hh_subset is not None else list(params.households.keys())
    C = list(params.crops.keys())
    L = list(params.livestock.keys())

    idx, nvars = build_index_maps(H, C, L)
    c = np.zeros(nvars)

    ym = scenario.yield_multiplier
    pm = scenario.crop_price_multiplier
    wm = scenario.wage_multiplier
    fm = scenario.fert_price_multiplier

    for h in H:
        c[idx[("hired", h, None)]] += (params.prices.wage * wm)
        c[idx[("off_farm", h, None)]] += -(params.prices.wage * wm)
        for cn in C:
            cp = params.crops[cn]
            c[idx[("sold", h, cn)]] += -(cp.price_sale * pm)
            per_ha_cost = cp.seed_cost_per_ha + fm * cp.fert_cost_per_ha + cp.chem_cost_per_ha
            c[idx[("area", h, cn)]] += per_ha_cost
        for l in L:
            lv = params.livestock[l]
            c[idx[("live_units", h, l)]] += -(lv.price_sale)
            c[idx[("live_units", h, l)]] += (lv.feed_cost_per_unit + lv.vet_cost_per_unit)

    A_eq, b_eq, A_ub, b_ub = [], [], [], []
    bounds = [(0, None) for _ in range(nvars)]

    for h in H:
        for cn in C:
            row = np.zeros(nvars)
            row[idx[("area", h, cn)]] = params.crops[cn].yield_per_ha * ym
            row[idx[("cons", h, cn)]] = -1.0
            row[idx[("sold", h, cn)]] = -1.0
            row[idx[("stored", h, cn)]] = -1.0
            A_eq.append(row); b_eq.append(0.0)

    for h in H:
        row = np.zeros(nvars)
        for cn in C: row[idx[("area", h, cn)]] = 1.0
        A_ub.append(row); b_ub.append(params.households[h].land_available)

    for h in H:
        row = np.zeros(nvars)
        for cn in C: row[idx[("area", h, cn)]] = params.crops[cn].labor_req_per_ha
        for l in L: row[idx[("live_units", h, l)]] = params.livestock[l].labor_req_per_unit
        row[idx[("off_farm", h, None)]] = 1.0
        row[idx[("hired", h, None)]] = -1.0
        A_ub.append(row); b_ub.append(params.households[h].labor_endowment)

    for h in H:
        row = np.zeros(nvars)
        row[idx[("hired", h, None)]] = 1.0
        A_ub.append(row); b_ub.append(params.households[h].max_hired_labor)

    for h in H:
        row = np.zeros(nvars)
        for cn in C: row[idx[("cons", h, cn)]] = -params.crops[cn].calorie_per_kg
        kcal_floor = (calories_floor_override if calories_floor_override is not None else params.min_kcal_per_person_per_day)
        kcal_need = kcal_floor * (params.households[h].adult_equiv * scenario.population_multiplier) * params.days_per_year
        A_ub.append(row); b_ub.append(-kcal_need)

    res = linprog(c, A_ub=np.array(A_ub), b_ub=np.array(b_ub), A_eq=np.array(A_eq), b_eq=np.array(b_eq), bounds=bounds, method='highs')

    out = {'success': bool(res.success), 'status': res.message, 'objective': None, 'by_household': {}}
    if not res.success:
        return out

    out['objective'] = -res.fun
    x = res.x
    def val(kind, h, k=None):
        return float(x[idx[(kind, h, k)]]) if (kind, h, k) in idx else 0.0

    for h in H:
        rev_crops = sum(val('sold', h, cn) * (params.crops[cn].price_sale * scenario.crop_price_multiplier) for cn in C)
        cost_crop_inputs = sum(val('area', h, cn) * (params.crops[cn].seed_cost_per_ha + scenario.fert_price_multiplier * params.crops[cn].fert_cost_per_ha + params.crops[cn].chem_cost_per_ha) for cn in C)
        rev_livestock = sum(val('live_units', h, l) * params.livestock[l].price_sale for l in L)
        cost_livestock = sum(val('live_units', h, l) * (params.livestock[l].feed_cost_per_unit + params.livestock[l].vet_cost_per_unit) for l in L)
        hired = val('hired', h)
        off = val('off_farm', h)
        rev_off = off * (params.prices.wage * scenario.wage_multiplier)
        cost_hired = hired * (params.prices.wage * scenario.wage_multiplier)

        out['by_household'][h] = {
            'revenue_breakdown': {'crops': rev_crops, 'livestock': rev_livestock, 'off_farm_wage': rev_off},
            'cost_breakdown': {'crop_inputs': cost_crop_inputs, 'livestock_costs': cost_livestock, 'hired_labor_cost': cost_hired},
        }

    return out

In [5]:
params, scenario = load_params_from_folder(Path('./data'))
demo_results = solve_single_year_lp(params, scenario, hh_subset=None)
print('Status:', demo_results['status'])
print('Success:', demo_results['success'])
print('Objective (net income):', demo_results['objective'])

# Flatten for preview (for validator)
if demo_results['success']:
    rows = []
    for h, hd in demo_results['by_household'].items():
        rows.append({
            'household_class': h,
            'rev_crops': hd['revenue_breakdown']['crops'],
            'rev_livestock': hd['revenue_breakdown']['livestock'],
            'rev_off_farm': hd['revenue_breakdown']['off_farm_wage'],
            'cost_crop_inputs': hd['cost_breakdown']['crop_inputs'],
            'cost_livestock': hd['cost_breakdown']['livestock_costs'],
            'cost_hired_labor': hd['cost_breakdown']['hired_labor_cost'],
        })
    model_df = pd.DataFrame(rows)
    try:
        tools.display_dataframe_to_user("Model summary (for validator)", model_df)
    except Exception:
        display(model_df)

with open('./outputs/demo_results.json', 'w') as f:
    json.dump(demo_results, f, indent=2)
print('Saved results to ./outputs/demo_results.json')

Status: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Success: True
Objective (net income): 851222.4841269841
Model summary (for validator)


Loading ITables v2.5.2 from the internet... (need help?)


Saved results to ./outputs/demo_results.json


## ✅ Validator: Compare Model vs Observed (Tables 10 & 11)
The file `./data/observed_table10_11.csv` contains values parsed from your tables (Birr/HH/year). Edit it anytime to try different observed targets.

In [6]:

model_path = Path('./outputs/demo_results.json')
if not model_path.exists():
    raise FileNotFoundError('Model results not found. Run the solver cell first to generate ./outputs/demo_results.json')
with open(model_path, 'r') as f:
    model = json.load(f)
if not model.get('success', False):
    raise RuntimeError('Model run was not successful. Please rerun the solver cell.')

rows = []
for h, hd in model['by_household'].items():
    rows.append({
        'household_class': h,
        'rev_crops': hd['revenue_breakdown']['crops'],
        'rev_livestock': hd['revenue_breakdown']['livestock'],
        'rev_off_farm': hd['revenue_breakdown']['off_farm_wage'],
        'cost_crop_inputs': hd['cost_breakdown']['crop_inputs'],
        'cost_livestock': hd['cost_breakdown']['livestock_costs'],
        'cost_hired_labor': hd['cost_breakdown']['hired_labor_cost'],
    })
model_df = pd.DataFrame(rows)
model_df['income_total'] = model_df['rev_crops'] + model_df['rev_livestock'] + model_df['rev_off_farm']
model_df['cost_total'] = model_df['cost_crop_inputs'] + model_df['cost_livestock'] + model_df['cost_hired_labor']

obs_df = pd.read_csv('./data/observed_table10_11.csv')
obs_df['income_total'] = obs_df['rev_crops'] + obs_df['rev_livestock'] + obs_df['rev_off_farm']
obs_df['cost_total'] = obs_df['cost_crop_inputs'] + obs_df['cost_livestock'] + obs_df['cost_hired_labor']

cols = ['rev_crops','rev_livestock','rev_off_farm','income_total','cost_crop_inputs','cost_livestock','cost_hired_labor','cost_total']
merged = model_df.merge(obs_df[['household_class']+cols], on='household_class', suffixes=('_model','_obs'), how='outer')

def pct_diff(m, o):
    if pd.isna(m) or pd.isna(o): return float('nan')
    if o == 0: return float('inf') if m != 0 else 0.0
    return 100.0 * (m - o) / abs(o)

rows_comp = []
for _, r in merged.iterrows():
    for k in cols:
        rows_comp.append({
            'household_class': r['household_class'],
            'metric': k,
            'observed': r[f'{k}_obs'],
            'model': r[f'{k}_model'],
            'diff': (r[f'{k}_model'] - r[f'{k}_obs']) if pd.notna(r[f'{k}_model']) and pd.notna(r[f'{k}_obs']) else float('nan'),
            'pct_diff_%': pct_diff(r[f'{k}_model'], r[f'{k}_obs'])
        })
comp_df = pd.DataFrame(rows_comp)

try:
    tools.display_dataframe_to_user('Validator — Model vs Observed (by metric)', comp_df)
except Exception:
    display(comp_df)

agg = []
for k in cols:
    sub = comp_df[comp_df['metric'] == k].copy()
    sub = sub.replace([np.inf, -np.inf], np.nan).dropna(subset=['observed','model'])
    if len(sub) == 0:
        rmse = mape = np.nan
    else:
        rmse = np.sqrt(np.mean((sub['model'] - sub['observed'])**2))
        mape = np.mean(np.abs((sub['model'] - sub['observed']) / sub['observed'].replace(0, np.nan))) * 100.0
    agg.append({'metric': k, 'RMSE': rmse, 'MAPE_%': mape})
agg_df = pd.DataFrame(agg)
try:
    tools.display_dataframe_to_user('Validator — Summary Errors (RMSE, MAPE%)', agg_df)
except Exception:
    display(agg_df)


Validator — Model vs Observed (by metric)


Loading ITables v2.5.2 from the internet... (need help?)


Validator — Summary Errors (RMSE, MAPE%)


Loading ITables v2.5.2 from the internet... (need help?)
